<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

# Create Player dataset

In [76]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

# Create custom projections
Maybe delete later when real projections available

In [77]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    elif pos == "K":
        base -= 3  # K-Penalty
    elif pos == "DST":
        base -= 3  # DST-Penalty
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,18.523900,20.097807,20.709649,21.389053,24.181327,21.271514,...,24.696341,0.000000,19.849602,24.662786,20.130439,22.502632,21.590743,20.589251,23.342928,22.181661
1,CeeDee Lamb,7,WR,2.6,22.897158,20.995333,22.538694,21.283326,25.291373,21.823827,...,21.242777,20.127886,17.551781,25.112130,21.340916,22.418917,22.381074,19.617828,19.430923,26.517128
2,Tyreek Hill,6,WR,3.2,21.117919,19.598210,20.153239,23.817572,20.101286,0.000000,...,18.190802,21.506399,24.665291,23.488147,20.291378,21.300177,23.158146,22.549000,24.519785,23.554848
3,Bijan Robinson,12,RB,5.0,20.026485,21.213259,21.088007,21.709714,17.573031,18.181232,...,22.730600,20.101896,21.013396,19.101779,0.000000,23.190244,20.699405,22.389218,21.549291,24.231835
4,Breece Hall,12,RB,5.4,20.982341,19.914922,20.705939,21.610697,21.869750,24.522369,...,22.096894,23.413690,23.326987,22.879104,0.000000,23.705214,24.390678,24.854872,17.297283,19.314582
5,Amon-Ra St. Brown,5,WR,6.2,20.061427,20.468980,22.781223,21.991327,0.000000,19.234156,...,22.655011,24.522861,22.551367,22.483599,23.935092,22.935789,17.821812,23.387279,21.442768,21.416256
6,Ja'Marr Chase,12,WR,6.6,20.958818,21.293115,19.745687,24.261473,20.890631,20.141048,...,19.455151,18.503061,24.332212,22.989377,0.000000,25.376857,19.181613,22.402165,21.697764,20.424976
7,Justin Jefferson,6,WR,7.0,23.293592,19.187460,22.330079,18.815178,22.405386,0.000000,...,19.963155,21.535754,20.494553,15.530515,22.078323,20.483857,18.307307,23.203823,18.226753,18.554362
8,Saquon Barkley,5,RB,9.2,21.043534,21.215014,18.910260,23.026153,0.000000,19.279549,...,18.816120,19.754591,17.524455,21.462880,23.404591,20.564203,20.963537,25.434569,21.718217,20.869146
9,A.J. Brown,5,WR,10.2,19.611102,19.842901,21.313396,21.193786,0.000000,20.464133,...,18.935379,20.342121,20.110768,20.193085,25.172700,22.769985,19.500812,20.862716,19.761868,17.754574


In [79]:
# --- 1. Parameter (aus deinem paste-2.txt) ---
players        = df_small['Player'].tolist()
positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks          = list(range(1,18))
pos            = dict(zip(df_small['Player'], df_small['POS']))
pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
beta           = {t:100.0 for t in weeks}
gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
alpha, lambda_0, lambda_1, lambda_2 = 1.0, 1, 100, 150
df_sorted = df_small.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
# Mindest‐Anzahl Spieler pro Position im Roster
min_pos_req = pos_limit


# --- 2. Teams, DM-Team und Snake-Draft ---
num_teams      = 12
teams          = [f"Team {i+1}" for i in range(num_teams)]
DM_team        = "Team 4"
num_rounds     = 15
draft_order    = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) verbleibende Spieler neu sortieren
    rem  = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) solange Defizite bestehen, aus allen rem dieser Position picken
    if needed:
        # Kandidaten aller benötigten Positionen
        candidates = [p for p in rem if pos[p] in needed]
        # begrenze auf Top-k, falls mehr Kandidaten vorhanden
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: zufällig aus Top-k aller Positionen
    return random.choice(rem[:topk])

# --- 3. Initialisierung ---
rosters        = {tm: [] for tm in teams}
available      = set(players)
draft_log      = []

# --- 4. Hauptschleife über alle Picks ---
for pick_idx, team in enumerate(draft_order, start=1):
    # 4.1 Ranking der verbleibenden Spieler aktualisieren
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
    Rk  = {p: i+1 for i,p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])


    # 4.2 DM-Pick via MIP
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # Variablen nur für verbleibende Spieler
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
              for i in player_vars}
        x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
             for i in rem for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # Fixiere vergangene Picks
        for p in rosters[team]:
            m += y[p] == 1

        # Objective
        m.objective = (
            lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
          + lambda_1 * xsum(z[t] for t in weeks[:15])
          + lambda_2 * xsum(z[t] for t in weeks[15:])
        )

        # Position‐ und Roster‐Constraints
        for j in positions:
            # Müssen mindestens gamma[j] ziehen
            m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
            for t in weeks:
                m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]
        # Roster Anforderungen
        for pos_name, req in min_pos_req.items():
            m += xsum(y[i] for i in player_vars if pos[i] == pos_name) >= req, f"min_{pos_name}"
        # maximale Picks pro Team
        m += xsum(y[i] for i in rem) <= num_rounds
        # nur gedraftete Spieler in der Week‐Lineup
        for i in rem:
            for t in weeks:
                m += x[i,t] <= y[i]
        # Win‐Indicator
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

        # Robuste Draft‐Constraint (1b) – grob implementiert
        n_k = pick_idx
        for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
            top_cut = int(alpha*(future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i,r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

        m.optimize()
        # gewählten Spieler extrahieren
        chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
    else:
        pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=0.01)

    # 4.4 Update
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx, "Team": team, "Player": pick,
        "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
    })

# --- 5. Ergebnis als DataFrame ---
df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1+num_teams*2))


    Pick     Team               Player  Round POS
0      1   Team 1  Christian McCaffrey      1  RB
1      2   Team 2          CeeDee Lamb      1  WR
2      3   Team 3       Bijan Robinson      1  RB
3      4   Team 4        Jake Ferguson      1  TE
4      5   Team 5          Tyreek Hill      1  WR
5      6   Team 6          Breece Hall      1  RB
6      7   Team 7    Amon-Ra St. Brown      1  WR
7      8   Team 8     Justin Jefferson      1  WR
8      9   Team 9        Ja'Marr Chase      1  WR
9     10  Team 10       Saquon Barkley      1  RB
10    11  Team 11      Jonathan Taylor      1  RB
11    12  Team 12       Garrett Wilson      1  WR
12    13  Team 12           A.J. Brown      2  WR
13    14  Team 11           Puka Nacua      2  WR
14    15  Team 10         Jahmyr Gibbs      2  RB
15    16   Team 9  Marvin Harrison Jr.      2  WR
16    17   Team 8       Kyren Williams      2  RB
17    18   Team 7        Derrick Henry      2  RB
18    19   Team 6   Travis Etienne Jr.      2  RB


In [80]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

POS,DST,K,QB,RB,TE,WR
Team,,,,,,
Team 1,1,1,3,4,2,4
Team 10,1,1,1,3,3,6
Team 11,1,1,2,3,1,7
Team 12,1,1,2,2,1,8
Team 2,1,1,2,3,2,6
Team 3,1,1,1,4,2,6
Team 4,0,0,2,6,3,4
Team 5,1,1,2,7,1,3
Team 6,1,1,2,5,1,5


In [81]:
result = {}

# Iteriere über jedes Team
for team in df_draft['Team'].unique():
    # Filtere alle Zeilen für das Team
    team_df = df_draft[df_draft['Team'] == team]

    # Sortiere nach 'Pick'
    team_df = team_df.sort_values(by='Pick')

    # Erzeuge Dictionary für Spieler des Teams
    player_dict = {
    row['Player']: (f"Round {row['Round']} Pick {row['Pick']}", row['POS'])
    for _, row in team_df.iterrows()
    }

    # Füge zum Gesamtergebnis hinzu
    result[team] = player_dict

# Beispielausgabe
result['Team 2']  # nun nach Pick sortiert

{'CeeDee Lamb': ('Round 1 Pick 2', 'WR'),
 'Davante Adams': ('Round 2 Pick 23', 'WR'),
 'Josh Jacobs': ('Round 3 Pick 26', 'RB'),
 'C.J. Stroud': ('Round 4 Pick 47', 'QB'),
 'Dalton Kincaid': ('Round 5 Pick 50', 'TE'),
 'David Montgomery': ('Round 6 Pick 71', 'RB'),
 'Brandon Aubrey': ('Round 7 Pick 74', 'K'),
 'Miami Dolphins': ('Round 8 Pick 95', 'DST'),
 'Malik Nabers': ('Round 9 Pick 98', 'WR'),
 'Jaylen Warren': ('Round 10 Pick 119', 'RB'),
 'Rome Odunze': ('Round 11 Pick 122', 'WR'),
 'Jameson Williams': ('Round 12 Pick 143', 'WR'),
 'Justin Herbert': ('Round 13 Pick 146', 'QB'),
 'Taysom Hill': ('Round 14 Pick 167', 'TE'),
 'Brandin Cooks': ('Round 15 Pick 170', 'WR')}